In [44]:
import pyodbc
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
model = pe.ConcreteModel("Cargacamiones")


In [45]:
path=r"C:\Users\juano\Documents\pyomcamiones\Carga de camiones\Transporte_cubicaje.mdb"
conn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+path+';')
cursor = conn.cursor()

Familias Carga de datos

In [46]:
cursor.execute('select IdProducto from Productos')
productos=[]
for row in cursor.fetchall():
    productos.append(row[0])

In [47]:
cursor.execute('select IdFamilia from Familias')
familias=[]
for row in cursor.fetchall():
    familias.append(row[0])

In [48]:
cursor.execute('select IdCamion from Camiones WHERE IdCamion<4')
camiones=[]
for i in range(10):
    camiones.append(i)

In [49]:
cursor.execute('select IdDia from Dias WHERE "IdDia"<=19')
dias=[]
for row in cursor.fetchall():
    dias.append(row[0])
        

In [50]:
cursor.execute('select IdDia from Dias')
dias1=[]
for row in cursor.fetchall():
    dias1.append(row[0])
    

## Familias Pyomo

In [51]:
model.i = pe.Set(initialize=productos)
model.f = pe.Set(initialize=familias)
model.c = pe.Set(initialize=camiones)
model.t = pe.Set(initialize=dias)
model.t1 = pe.Set(initialize=dias1)

## Carga de datos productos

In [52]:
cursor.execute('select IdFamilia, LotePedido from Familias')
lotepedido={}
for row in cursor.fetchall():
    lotepedido.update({row[0]:row[1]})


In [53]:
cursor.execute('select IdProducto , PiezasCont from Productos')
piezascont={}
for row in cursor.fetchall():
    piezascont.update({row[0]:row[1]})

In [54]:
cursor.execute('select IdProducto , IdFamilia, beta from beta')
b={}
for row in cursor.fetchall():
    b.update({(row[0],row[1]):row[2]})


In [55]:
cursor.execute('select IdProducto , StockInicial from Productos')
stockInicial={}
for row in cursor.fetchall():
    stockInicial.update({row[0]:row[1]})


In [56]:
cursor.execute('select  IdProducto , IdDia, Demanda from Demanda')
demanda={}
for row in cursor.fetchall():
    demanda.update({(row[0],row[1]):row[2]})

In [57]:
cursor.execute('select  IdProducto , Valor from Productos')
valor={}
for row in cursor.fetchall():
    valor.update({row[0]:row[1]})

In [58]:
PC={}
for camion in camiones:
    PC.update({camion:1000})

In [59]:
L=78
nu=26/30
NT_lw=5
NT_up=10

In [60]:
model.LotePedido = pe.Param(model.f , initialize=lotepedido)
model.PiezasCont= pe.Param(model.i ,initialize=piezascont)
model.b= pe.Param(model.i, model.f ,initialize=b, default=0)
model.StockInicial= pe.Param(model.i, initialize=stockInicial)
model.demanda= pe.Param(model.i, model.t1,  initialize=demanda)
model.valor=pe.Param(model.i, initialize=stockInicial)
model.PC=pe.Param(model.c, initialize=PC)
model.L=pe.Param(initialize=L)
model.nu=pe.Param(initialize=nu)
model.NT_lw=pe.Param(initialize=NT_lw)
model.NT_up=pe.Param(initialize=NT_up)

# # Varibles

In [61]:
model.CantidadPedir = pe.Var(model.i, model.c, model.t, domain=pe.Integers)

In [62]:
model.CantidadPedir2 = pe.Var(model.i,model.f, model.c, model.t)
model.ContenedoresCamion=pe.Var(model.c,model.t, domain=pe.Integers)

In [63]:
model.Stock=pe.Var(model.i,model.t, bounds=(0,None))

In [64]:
model.RetrasoDemanda=pe.Var(model.i,model.t)

In [65]:
model.K=pe.Var(model.f,model.c,model.t, domain=pe.Integers)

In [66]:
model.Y=pe.Var(model.c,model.t ,domain=pe.Binary, initialize=0)

## Función Objetivo

In [67]:
expr = sum(model.PC[c]*model.Y[c,t]  for c in model.c for t in model.t)

In [68]:
 #Coste Transporte
model.objective = pe.Objective(sense=pe.minimize, expr=expr)

## Funciones auxiliares

In [69]:
def UsoCamion(model):
    return sum(model.L*model.Y[c,t] for c in model.c for t in model.t)-\
        sum(model.ContenedoresCamion[c,t] for c in model.c for t in model.t)

In [70]:
def CTransporte(model):
    return sum(model.PC[c]*model.Y[c,t] for c in model.c for t in model.t)

In [71]:
def NumeroCamiones(model):
    return sum(model.Y[c,t] for c in model.c for t in model.t)

## Restricciones

In [72]:
model.stocks=pe.ConstraintList()

In [73]:
for i in model.i:
    lhs=model.Stock[i,1]
    rhs=model.StockInicial[i]-model.demanda[i,1]+sum(model.CantidadPedir[i,c,1] for c in model.c)
    model.stocks.add(lhs==rhs)

In [74]:
for t in list(model.t)[1:]:
    for i in model.i:
        lhs=model.Stock[i,t]
        rhs=model.StockInicial[i]-model.demanda[i,t-1]+sum(model.CantidadPedir[i,c,t-1] for c in model.c)
        model.stocks.add(lhs==rhs)

In [75]:
model.RcantidadPedir=pe.ConstraintList()

In [76]:
for i in model.i:
    for f in model.f:
        for c in model.c:
            for t in model.t:
                lhs=model.CantidadPedir2[i,f,c,t]
                rhs=model.K[f,c,t]*model.LotePedido[f]*model.b[i,f]
                model.RcantidadPedir.add(lhs==rhs)

In [77]:
model.RcantidadPedir2=pe.ConstraintList()

In [78]:
for i in model.i:
    for c in model.c:
        for t in model.t:
            lhs=model.CantidadPedir[i,c,t]
            rhs=sum(model.CantidadPedir2[i,f,c,t] for f in model.f)
            model.RcantidadPedir2.add(lhs==rhs)

In [79]:
model.RContendores=pe.ConstraintList()

In [80]:
for c in model.c:
    for t in model.t:
        lhs=model.ContenedoresCamion[c,t]
        rhs=sum(model.CantidadPedir[i,c,t]/model.PiezasCont[i] for i in model.i)
        model.RContendores.add(lhs==rhs)

In [81]:
model.RDimensionesCamion=pe.ConstraintList()

In [82]:
for c in model.c:
    for t in model.t:
        lhs=model.ContenedoresCamion[c,t]
        rhs=model.L*model.Y[c,t]
        model.RDimensionesCamion.add(lhs<=rhs)

In [83]:
model.RLlenado=pe.ConstraintList()
for c in model.c:
    for t in model.t:
        lhs=model.ContenedoresCamion[c,t]
        rhs=model.L*model.nu*model.Y[c,t]
        model.RLlenado.add(lhs>=rhs)

In [84]:
model.RCobertura=pe.ConstraintList()
for i in model.i:
    for t in model.t:
        lhs=model.Stock[i,t]
        rhs=model.demanda[i,t+1]*3
        model.RCobertura.add(lhs>=rhs)

In [85]:
model.bounds=pe.ConstraintList()
for i in model.i:
    for t in model.t:
        model.bounds.add(model.Stock[i,t]>=0)


In [86]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-06
Read LP format model from file C:\Users\juano\AppData\Local\Temp\tmp3s1dsq3g.pyomo.lp
Reading time = 0.13 seconds
x73816: 70111 rows, 73531 columns, 147631 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 70111 rows, 73531 columns and 147631 nonzeros
Model fingerprint: 0xdc2bf6ee
Variable types: 64411 continuous, 9120 integer (285 binary)
Coefficient statistics:
  Matrix range     [2e-02, 8e+01]
  Objective range  [1e+03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 69031 rows and 68941 columns
Presolve time: 0.24s
Presolved: 1080 rows, 4590 columns, 9450 nonzeros
Variable types: 0 continuous, 4590 integer (270 binary)

Root relaxation: objective 5.973179e+04, 758 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node